In [44]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import plotly.graph_objects as go

import random
import pandas as pd
import numpy as np

!pip install torchinfo
from torchinfo import summary

In [45]:
# Random Seed 고정 (학습 반복 시행 시에도 동일한 결과가 나오도록)

seed = 20250305

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

In [46]:
# line 을 보다 길게 표시 + 지수 표현이 아닌 원래 숫자 표현으로

torch.set_printoptions(linewidth=160, sci_mode=False)

**1. 데이터셋 로딩 및 데이터 분석**

In [47]:
# 데이터셋 로딩

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = torchvision.datasets.MNIST(root='./data',
                                           train=True,
                                           transform=transform,
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='./data',
                                          train=False,
                                          transform=transform,
                                          download=True)


In [48]:
# 시간 절약을 위해, 학습 데이터에서 랜덤하게 일부 샘플만 추출

from torch.utils.data import Subset, DataLoader

NUM_TRAIN_SAMPLES = 5000
BATCH_SIZE = 32

subset_indices = random.sample(range(len(train_dataset)), NUM_TRAIN_SAMPLES)
train_subset = Subset(train_dataset, subset_indices)

train_loader = DataLoader(train_subset,
                          batch_size=BATCH_SIZE,
                          shuffle=True)

# 테스트 데이터셋은 학습 대상이 아니므로 그대로 이용
test_loader = DataLoader(test_dataset,
                         batch_size=BATCH_SIZE,
                         shuffle=False)

In [49]:
# 클래스 불균형 분석

# 학습 데이터
train_labels = torch.tensor([train_subset.dataset.targets[i] for i in subset_indices])
train_class_counts = torch.bincount(train_labels)
print(train_class_counts)

NUM_CLASSES = len(train_class_counts)

tensor([474, 555, 537, 545, 505, 422, 485, 492, 478, 507])


In [50]:
train_class_percentage = np.array(train_class_counts) * 100.0 / sum(train_class_counts)

train_y_distrib = pd.DataFrame({'class': list(range(NUM_CLASSES)),
                                'count': train_class_counts,
                                'percentage (%)': train_class_percentage})

train_y_distrib

,class,count,percentage (%)
0,0,474,9.48
1,1,555,11.10
2,2,537,10.74
3,3,545,10.90
4,4,505,10.10
5,5,422,8.44
6,6,485,9.70
7,7,492,9.84
8,8,478,9.56
9,9,507,10.14


In [51]:
# 테스트 데이터
test_labels = test_loader.dataset.targets
test_class_counts = torch.bincount(test_labels)
print(test_class_counts)

tensor([ 980, 1135, 1032, 1010,  982,  892,  958, 1028,  974, 1009])


In [52]:
test_class_percentage = np.array(test_class_counts) * 100.0 / sum(test_class_counts)

test_y_distrib = pd.DataFrame({'class': list(range(NUM_CLASSES)),
                               'count': test_class_counts,
                               'percentage (%)': test_class_percentage})

test_y_distrib

,class,count,percentage (%)
0,0,980,9.80
1,1,1135,11.35
2,2,1032,10.32
3,3,1010,10.10
4,4,982,9.82
5,5,892,8.92
6,6,958,9.58
7,7,1028,10.28
8,8,974,9.74
9,9,1009,10.09


**2. CNN 모델 정의**

In [53]:
# CNN 모델 정의

class CNN(nn.Module):

    def __init__(self, num_classes=10, final_activation=nn.Softmax()):
        super(CNN, self).__init__()

        # Conv
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.ReLU()
        )
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Sequential(
            nn.Conv2d(64, 64, kernel_size=3),
            nn.ReLU()
        )

        # Fully Connected
        self.fc1 = nn.Sequential(
            nn.Linear(64 * 4 * 4, 64),
            nn.Sigmoid()
        )

        if final_activation is not None:
            self.fc_final = nn.Sequential(
                nn.Linear(64, num_classes),
                final_activation
            )
        else:
            self.fc_final = nn.Linear(64, num_classes)

    def forward(self, x):

        # Conv
        x = self.conv1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)

        x = x.view(-1, 64 * 4 * 4)

        # Fully Connected
        x = self.fc1(x)
        x = self.fc_final(x)

        return x

In [54]:
# 모델 구조 출력

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)

print(summary(model, input_size=(BATCH_SIZE, 1, 28, 28)))

Layer (type:depth-idx)                   Output Shape              Param #
CNN                                      [32, 10]                  --
├─Sequential: 1-1                        [32, 32, 28, 28]          --
│    └─Conv2d: 2-1                       [32, 32, 28, 28]          320
│    └─ReLU: 2-2                         [32, 32, 28, 28]          --
├─MaxPool2d: 1-2                         [32, 32, 14, 14]          --
├─Sequential: 1-3                        [32, 64, 12, 12]          --
│    └─Conv2d: 2-3                       [32, 64, 12, 12]          18,496
│    └─ReLU: 2-4                         [32, 64, 12, 12]          --
├─MaxPool2d: 1-4                         [32, 64, 6, 6]            --
├─Sequential: 1-5                        [32, 64, 4, 4]            --
│    └─Conv2d: 2-5                       [32, 64, 4, 4]            36,928
│    └─ReLU: 2-6                         [32, 64, 4, 4]            --
├─Sequential: 1-6                        [32, 64]                  --
│    └

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1844: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return inner()


**3. 데이터셋 분리**

* Train Data -> Train Data + Valid Data

In [55]:
# 데이터셋 분리

from torch.utils.data import random_split

# 샘플 수
num_train = 2000
num_valid = 3000

assert NUM_TRAIN_SAMPLES == num_train + num_valid

# 데이터셋 분리
train_dataset, valid_dataset =\
    random_split(train_subset, [num_train, num_valid])

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=False)

**4. 학습 실시 함수**

In [56]:
MAX_EPOCHS = 65536
EARLY_STOPPING_ROUNDS = 10  # Early Stopping Patience (epochs)

In [57]:
from sklearn.metrics import accuracy_score
from copy import deepcopy

In [58]:
# 모델 학습 실시

# args :
# - model           : 학습할 모델
# - train_loader    : Training Data Loader
# - train_loss_list : 각 epoch 에서의 train loss 기록

# returns :
# - train_loss : 모델의 Train Loss

def run_train(model, train_loader, train_loss_list):
    model.train()
    train_loss = 0.0
    cnt = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        # train 실시
        model.optimizer.zero_grad()
        outputs = model(images)

        # Regression or Prob. Prediction 일 때 처리
        if model.task_type in ['regression', 'prob_prediction']:
            outputs = torch.flatten(outputs).float()
            labels = labels.float()

        # Classification - Multi Label 일 때 처리
        elif model.multi_label:
            pass

        # Classification - Single Label 일 때
        # Loss Function 이 Binary Cross Entropy / Mean Squared Error 인 경우 output shape 변경 처리
        elif model.loss_function.__class__.__name__ in ['BCELoss', 'BCEWithLogitsLoss', 'MSELoss']:
            labels = torch.nn.functional.one_hot(labels,
                                                 num_classes=model.num_classes)
            labels = labels.float()

        # Loss 처리
        loss = model.loss_function(outputs, labels)
        loss.backward()
        model.optimizer.step()

        train_loss += loss.item()
        cnt += 1

    train_loss_list.append(train_loss / len(train_loader))
    return train_loss_list[-1]

In [59]:
# 모델 validation 실시

# args :
# - model                  : validation 할 모델
# - valid_loader           : Validation Data Loader
# - normalized_output      : dataset 의 출력값이 normalize 되었는지 여부
# - print_first_batch_info : 1번째 batch 에서 output, label 을 출력할지 여부

# returns :
# - accuracy : 모델의 validation 정확도 (Classification or Prob. Prediction 인 경우)
# - mse      : 모델의 validation MSE (Regression 인 경우)

def run_validation(model, valid_loader, normalized_output=False, print_first_batch_info=False):
    model.eval()
    correct, total = 0, 0
    squared_error_sum = 0.0
    print_output_and_label = True

    with torch.no_grad():
        for images, labels in valid_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)

            # 정규화 처리
            if normalized_output:
                outputs = outputs * model.train_output_std + model.train_output_mean
                labels = labels * model.train_output_std + model.train_output_mean

            # 첫번째 batch 의 output 및 label 출력
            if print_first_batch_info and print_output_and_label:
                print(f'[ outputs ]\n{outputs}\n[ labels ]\n{labels}')
                print_output_and_label = False

            # validation 실시 및 정확도 측정
            total += labels.size(0)

            # For Regression
            if model.task_type == 'regression':
                outputs = torch.flatten(outputs).float()
                labels = torch.flatten(labels).float()
                squared_error_sum += nn.MSELoss()(outputs, labels) * labels.size(0)

            # For Probability Prediction
            elif model.task_type == 'prob_prediction':
                outputs_hard = torch.round(outputs)
                outputs_hard = torch.flatten(outputs_hard)
                correct += (outputs_hard == labels).sum().item()

            # For Multi Label Classification
            # (각 sample 당 존재하는 num_classes 개의 0 ~ 1 값을 prediction 으로 간주하고,
            #  모든 sample 에 대한 전체 prediction 중 정답 prediction 의 비율로 정확도 측정)
            elif model.multi_label:
                outputs_hard = torch.round(outputs)
                correct += (outputs_hard == labels).sum().item() / model.num_classes

            # For Single Label Classification
            else:
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == labels).sum().item()

    if model.task_type == 'regression':
        mse = squared_error_sum / total
        return mse

    else:
        accuracy = correct / total
        return accuracy

In [60]:
# metric 이름 반환

def get_metric_name(task_type):
    if task_type == 'regression':
        return 'MSE'
    else:
        return 'Accuracy'

In [61]:
# Regression 정규화 시 학습 데이터 출력값의 평균, 표준편차 정보 저장

# args :
# - model        : 학습할 모델
# - train_loader : Training Data Loader

# returns :
# - model.train_output_mean 에 Training Data output 의 평균 저장
# - model.train_output_std 에 Training Data output 의 표준편차 저장

def store_train_output_mean_std(model, train_loader):
    output_list = []

    for inputs, labels in train_loader:
        output_list += list(labels.cpu().numpy())

    model.train_output_mean = np.mean(output_list)
    model.train_output_std = np.std(output_list)

In [62]:
from torch.utils.data import Dataset

class NormalizedDataset(Dataset):
    def __init__(self, dataset, mean, std):
        self.dataset = dataset
        self.mean = mean
        self.std = std

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        x, y = self.dataset[idx]
        y_standardized = (y - self.mean) / self.std
        return x, y_standardized


# DataLoader 의 모든 출력값을 Normalize 처리

# args :
# - data_loader : 모든 출력값을 Normalize 처리할 DataLoader
# - mean        : 평균값
# - std         : 표준편차
# - shuffle     : shuffle 여부 (train 에서만 True)

def generate_normalized_dataloader(data_loader, mean, std, shuffle):
    normalized_dataset = NormalizedDataset(data_loader.dataset, mean, std)
    new_dataloader = DataLoader(normalized_dataset, batch_size=BATCH_SIZE, shuffle=shuffle)

    return new_dataloader

In [63]:
# 모델 학습 및 validation 전체 프로세스

# args :
# - model        : 학습할 모델
# - train_loader : Training Data Loader
# - valid_loader : 각 epoch 마다 validation 할 Valid Data Loader
# - test_loader  : 최종적으로 성능을 평가할 Test Data Loader
# - verbose      : 학습 중 프로세스 출력 여부

# returns :
# - final_metric     : 해당 하이퍼파라미터 조합에 대한 최종 성능지표 값 (valid metric 이 가장 좋았던 epoch 의 모델로 측정)
# - best_epoch_model : valid metric 이 가장 좋았던 epoch 에서 생성된 모델

def run_model_common(model, train_loader, valid_loader, test_loader, verbose=False):

    train_loss_list = []          # train loss
    valid_metric_list = []        # valid metric (Accuracy for Classification, MSE for Regression)
    best_valid_metric = None      # best validation accuracy
    best_valid_metric_epoch = -1  # valid metric 이 가장 좋았던 epoch
    best_epoch_model = None       # valid metric 이 가장 좋았던 epoch 의 모델

    metric_name = get_metric_name(model.task_type)
    do_normalize = model.additional_options.get('normalization', False) is True

    # 평균 및 표준편차 저장 및 표준화된 Data Loader 생성
    if do_normalize:
        store_train_output_mean_std(model, train_loader)
        print(f'train output mean: {model.train_output_mean}, std: {model.train_output_std}')

        train_loader_norm = generate_normalized_dataloader(data_loader=train_loader,
                                                           mean=model.train_output_mean,
                                                           std=model.train_output_std,
                                                           shuffle=True)

        valid_loader_norm = generate_normalized_dataloader(data_loader=valid_loader,
                                                           mean=model.train_output_mean,
                                                           std=model.train_output_std,
                                                           shuffle=False)

        test_loader_norm = generate_normalized_dataloader(data_loader=test_loader,
                                                          mean=model.train_output_mean,
                                                          std=model.train_output_std,
                                                          shuffle=False)

        train_loader_ = train_loader_norm
        valid_loader_ = valid_loader_norm
        test_loader_ = test_loader_norm

    else:
        train_loader_ = train_loader
        valid_loader_ = valid_loader
        test_loader_ = test_loader

    # 1. 학습 실시
    for epoch in range(MAX_EPOCHS):

        # 1-1. train model
        train_loss = run_train(model, train_loader_, train_loss_list)

        # 1-2. validate model (with EPOCH VALID SET)
        epoch_metric = run_validation(model=model,
                                      valid_loader=valid_loader_,
                                      normalized_output=do_normalize)

        valid_metric_list.append(epoch_metric)

        # 1-3. Early Stopping 처리 (overfitting 방지)
        if epoch == 0:
            improved = True  # 1st epoch 에서는 항상 improve 되었다고 가정
        elif model.task_type == 'regression':
            improved = epoch_metric < best_valid_metric
        else:
            improved = epoch_metric > best_valid_metric

        if improved:
            best_valid_metric = epoch_metric
            best_valid_metric_epoch = epoch

            best_epoch_model = CNN(num_classes=model.num_classes,
                                   final_activation=model.final_activation).to(device)

            best_epoch_model.multi_label = model.multi_label
            best_epoch_model.num_classes = model.num_classes
            best_epoch_model.task_type = model.task_type
            best_epoch_model.additional_options = model.additional_options

            if do_normalize:
                best_epoch_model.train_output_mean = model.train_output_mean
                best_epoch_model.train_output_std = model.train_output_std

            best_epoch_model.load_state_dict(model.state_dict())

            if verbose:
                print('best model updated')

        if epoch - best_valid_metric_epoch >= EARLY_STOPPING_ROUNDS:
            total_epochs = epoch
            break

        # 1-4. 결과 출력
        if verbose:
            print(f"Epoch {epoch+1}, Train Loss: {train_loss:.4f}, Valid {metric_name}: {epoch_metric:.4f}")

    # check best-epoch model correctly loaded
    checked_metric = run_validation(model=best_epoch_model,
                                    valid_loader=valid_loader_,
                                    normalized_output=do_normalize)

    if verbose:
        print(f"Best Epoch: {best_valid_metric_epoch}, Best Valid {metric_name}: {best_valid_metric}")
        print(f"Valid {metric_name} (with VALID set) on Loaded Best Model: {checked_metric}")

    assert abs(best_valid_metric - checked_metric) < 1e-8

    # 2. validate best-epoch model (with TEST SET)
    final_metric = run_validation(model=best_epoch_model,
                                  valid_loader=test_loader_,
                                  normalized_output=do_normalize,
                                  print_first_batch_info=True)

    if verbose:
        print(f"Final {metric_name} (with TEST set) on Loaded Best Model: {final_metric}")

    return final_metric, best_epoch_model, total_epochs

In [64]:
print(device)

cuda


**5. 실험 실시**

In [65]:
# 실험 실시

# args:
# - loss_function      : 모델 학습에 사용할 Loss Function
# - train_loader       : Training Data Loader
# - valid_loader       : 각 epoch 마다 validation 할 Valid Data Loader
# - test_loader        : 최종적으로 성능을 평가할 Test Data Loader
# - num_classes        : 모델이 분류할 class 개수 (또는 output value 개수)
# - task_type          : 'classification', 'regression' or 'prob_prediction'
# - final_activation   : 모델의 최종 Layer 의 Activation Function
# - multi_label        : Multi-Label 여부
# - additional_options : 모델 configuration 추가 옵션

# returns:
# - final_metric     : Test dataset 성능지표 값
# - best_epoch_model : Valid dataset 성능지표 값이 가장 높은 모델

def run_experiment(loss_function, train_loader, valid_loader, test_loader,
                   num_classes=10, task_type='classification',
                   final_activation=nn.Softmax(), multi_label=False,
                   additional_options={}):

    # define and run model
    model = CNN(num_classes, final_activation).to(device)

    # model configurations
    model.num_classes = num_classes
    model.final_activation = final_activation
    model.loss_function = loss_function
    model.optimizer = torch.optim.AdamW(model.parameters(), lr=0.001)
    model.multi_label = multi_label
    model.task_type = task_type
    model.additional_options = additional_options

#    print(summary(model, input_size=(BATCH_SIZE, 1, 28, 28)))

    final_metric, best_epoch_model, total_epochs = run_model_common(model,
                                                                    train_loader,
                                                                    valid_loader,
                                                                    test_loader,
                                                                    verbose=True)

    metric_name = get_metric_name(model.task_type)
    print(f"{metric_name}: {final_metric:.4f}, Total Epochs: {total_epochs}")

    return final_metric, best_epoch_model

In [66]:
# 실험 함수의 정상 동작 여부 확인

base_loss_function = nn.CrossEntropyLoss()
_, _ = run_experiment(base_loss_function, train_loader, valid_loader, test_loader)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 2.1659, Valid Accuracy: 0.5947
best model updated
Epoch 2, Train Loss: 1.9115, Valid Accuracy: 0.7900
best model updated
Epoch 3, Train Loss: 1.7405, Valid Accuracy: 0.8587
best model updated
Epoch 4, Train Loss: 1.6525, Valid Accuracy: 0.8690
best model updated
Epoch 5, Train Loss: 1.6122, Valid Accuracy: 0.8713
best model updated
Epoch 6, Train Loss: 1.5926, Valid Accuracy: 0.8833
best model updated
Epoch 7, Train Loss: 1.5637, Valid Accuracy: 0.9533
best model updated
Epoch 8, Train Loss: 1.5176, Valid Accuracy: 0.9593
best model updated
Epoch 9, Train Loss: 1.4979, Valid Accuracy: 0.9670
Epoch 10, Train Loss: 1.4891, Valid Accuracy: 0.9660
best model updated
Epoch 11, Train Loss: 1.4824, Valid Accuracy: 0.9677
Epoch 12, Train Loss: 1.4766, Valid Accuracy: 0.9653
best model updated
Epoch 13, Train Loss: 1.4737, Valid Accuracy: 0.9693
Epoch 14, Train Loss: 1.4721, Valid Accuracy: 0.9690
Epoch 15, Train Loss: 1.4710, Valid Accuracy: 0.9690
Epoch

In [67]:
# RMSE Loss 가 Pytorch 에 없으므로 따로 정의
# Source   : https://discuss.pytorch.org/t/rmse-loss-function/16540/4, by YannDubs1 (Yann Dubois)
# Modified : spacings, eps = 1e-6 -> 1e-8

class RMSELoss(nn.Module):
    def __init__(self, eps=1e-8):
        super().__init__()
        self.mse = nn.MSELoss()
        self.eps = eps

    def forward(self, yhat, y):
        loss = torch.sqrt(self.mse(yhat, y) + self.eps)
        return loss

5-1. Regression

In [68]:
# Regression 용으로 변경

from torch.utils.data import Dataset

class MNIST_reg(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.regression_map = {1: 1.0,
                               0: 2.0, 2: 2.0, 3: 2.0, 7: 2.0,
                               5: 3.0, 6: 3.0, 9: 3.0,
                               8: 4.0,
                               4: 6.0}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        new_label = self.regression_map[label]
        return image, new_label

In [69]:
# 데이터셋 클래스 분포 분석 함수

def analyze_class_distribution(data_loader):
    targets = [data_loader.dataset[i][1] for i in range(len(data_loader.dataset))]

    class_list, class_counts = np.unique(targets, return_counts=True)
    class_percentage = np.array(class_counts) * 100.0 / sum(class_counts)

    class_distrib = pd.DataFrame({'class': class_list,
                                  'count': class_counts,
                                  'percentage (%)': class_percentage})

    return class_distrib

In [70]:
# Regression 용 Data Loader

train_loader_reg = DataLoader(MNIST_reg(train_loader.dataset),
                              batch_size=BATCH_SIZE,
                              shuffle=True)

valid_loader_reg = DataLoader(MNIST_reg(valid_loader.dataset),
                              batch_size=BATCH_SIZE,
                              shuffle=False)

test_loader_reg = DataLoader(MNIST_reg(test_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=False)

In [71]:
analyze_class_distribution(train_loader_reg)

,class,count,percentage (%)
0,1.0,239,11.95
1,2.0,795,39.75
2,3.0,585,29.25
3,4.0,181,9.05
4,6.0,200,10.00


In [72]:
analyze_class_distribution(valid_loader_reg)

,class,count,percentage (%)
0,1.0,316,10.533333
1,2.0,1253,41.766667
2,3.0,829,27.633333
3,4.0,297,9.900000
4,6.0,305,10.166667


In [73]:
analyze_class_distribution(test_loader_reg)

,class,count,percentage (%)
0,1.0,1135,11.35
1,2.0,4050,40.50
2,3.0,2859,28.59
3,4.0,974,9.74
4,6.0,982,9.82


In [74]:
# Regression (정규화 미 적용 시) 실험 진행

_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_reg,
                                     valid_loader=valid_loader_reg,
                                     test_loader=test_loader_reg,
                                     num_classes=1,
                                     task_type='regression',
                                     additional_options={'normalization': True},
                                     final_activation=None)

train output mean: 2.754, std: 1.343682998329591
best model updated
Epoch 1, Train Loss: 0.5569, Valid MSE: 0.5819
best model updated
Epoch 2, Train Loss: 0.2585, Valid MSE: 0.2765
best model updated
Epoch 3, Train Loss: 0.1384, Valid MSE: 0.2099
best model updated
Epoch 4, Train Loss: 0.0991, Valid MSE: 0.1914
best model updated
Epoch 5, Train Loss: 0.0804, Valid MSE: 0.1646
best model updated
Epoch 6, Train Loss: 0.0653, Valid MSE: 0.1489
best model updated
Epoch 7, Train Loss: 0.0592, Valid MSE: 0.1453
best model updated
Epoch 8, Train Loss: 0.0408, Valid MSE: 0.1336
best model updated
Epoch 9, Train Loss: 0.0318, Valid MSE: 0.1166
best model updated
Epoch 10, Train Loss: 0.0258, Valid MSE: 0.1066
best model updated
Epoch 11, Train Loss: 0.0218, Valid MSE: 0.1033
best model updated
Epoch 12, Train Loss: 0.0169, Valid MSE: 0.0990
Epoch 13, Train Loss: 0.0156, Valid MSE: 0.1042
best model updated
Epoch 14, Train Loss: 0.0114, Valid MSE: 0.0883
Epoch 15, Train Loss: 0.0087, Valid MSE: 

In [75]:
_, best_epoch_model = run_experiment(nn.L1Loss(),
                                     train_loader=train_loader_reg,
                                     valid_loader=valid_loader_reg,
                                     test_loader=test_loader_reg,
                                     num_classes=1,
                                     task_type='regression',
                                     additional_options={'normalization': True},
                                     final_activation=None)

train output mean: 2.754, std: 1.343682998329591
best model updated
Epoch 1, Train Loss: 0.5384, Valid MSE: 0.7558
best model updated
Epoch 2, Train Loss: 0.3882, Valid MSE: 0.4465
best model updated
Epoch 3, Train Loss: 0.2973, Valid MSE: 0.2548
best model updated
Epoch 4, Train Loss: 0.2245, Valid MSE: 0.2068
best model updated
Epoch 5, Train Loss: 0.1903, Valid MSE: 0.1716
best model updated
Epoch 6, Train Loss: 0.1654, Valid MSE: 0.1533
best model updated
Epoch 7, Train Loss: 0.1521, Valid MSE: 0.1359
best model updated
Epoch 8, Train Loss: 0.1380, Valid MSE: 0.1348
Epoch 9, Train Loss: 0.1260, Valid MSE: 0.1454
best model updated
Epoch 10, Train Loss: 0.1168, Valid MSE: 0.1312
Epoch 11, Train Loss: 0.1072, Valid MSE: 0.1334
best model updated
Epoch 12, Train Loss: 0.0965, Valid MSE: 0.1156
best model updated
Epoch 13, Train Loss: 0.0863, Valid MSE: 0.1051
Epoch 14, Train Loss: 0.0765, Valid MSE: 0.1084
best model updated
Epoch 15, Train Loss: 0.0673, Valid MSE: 0.1037
best model u

In [76]:
_, best_epoch_model = run_experiment(RMSELoss(),
                                     train_loader=train_loader_reg,
                                     valid_loader=valid_loader_reg,
                                     test_loader=test_loader_reg,
                                     num_classes=1,
                                     task_type='regression',
                                     additional_options={'normalization': True},
                                     final_activation=None)

train output mean: 2.754, std: 1.343682998329591
best model updated
Epoch 1, Train Loss: 0.7765, Valid MSE: 0.5916
best model updated
Epoch 2, Train Loss: 0.4858, Valid MSE: 0.2485
best model updated
Epoch 3, Train Loss: 0.3495, Valid MSE: 0.1842
best model updated
Epoch 4, Train Loss: 0.2875, Valid MSE: 0.1428
best model updated
Epoch 5, Train Loss: 0.2469, Valid MSE: 0.1210
best model updated
Epoch 6, Train Loss: 0.2097, Valid MSE: 0.1021
best model updated
Epoch 7, Train Loss: 0.1868, Valid MSE: 0.0946
Epoch 8, Train Loss: 0.1422, Valid MSE: 0.1212
Epoch 9, Train Loss: 0.1284, Valid MSE: 0.0982
best model updated
Epoch 10, Train Loss: 0.0914, Valid MSE: 0.0814
Epoch 11, Train Loss: 0.0831, Valid MSE: 0.0999
best model updated
Epoch 12, Train Loss: 0.0676, Valid MSE: 0.0772
Epoch 13, Train Loss: 0.0645, Valid MSE: 0.0786
Epoch 14, Train Loss: 0.0485, Valid MSE: 0.0775
Epoch 15, Train Loss: 0.0433, Valid MSE: 0.0797
Epoch 16, Train Loss: 0.0409, Valid MSE: 0.0796
Epoch 17, Train Loss:

In [77]:
"""
CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.
"""

# try:
#     _, best_epoch_model = run_experiment(nn.BCELoss(),
#                                         train_loader=train_loader_reg,
#                                         valid_loader=valid_loader_reg,
#                                         test_loader=test_loader_reg,
#                                         num_classes=1,
#                                         task_type='regression',
#                                         additional_options={'normalization': True},
#                                         final_activation=None)
#
# except Exception as e:
#     print(e)

'\nCUDA error: device-side assert triggered\nCUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.\nFor debugging consider passing CUDA_LAUNCH_BLOCKING=1\nCompile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.\n'

In [78]:
try:
    _, best_epoch_model = run_experiment(nn.BCEWithLogitsLoss(),
                                        train_loader=train_loader_reg,
                                        valid_loader=valid_loader_reg,
                                        test_loader=test_loader_reg,
                                        num_classes=1,
                                        task_type='regression',
                                        additional_options={'normalization': True},
                                        final_activation=None)

except Exception as e:
    print(e)

train output mean: 2.754, std: 1.343682998329591
best model updated
Epoch 1, Train Loss: 0.1350, Valid MSE: 15.8230
Epoch 2, Train Loss: 0.0508, Valid MSE: 20.5846
Epoch 3, Train Loss: 0.0288, Valid MSE: 25.0897
Epoch 4, Train Loss: 0.0228, Valid MSE: 28.0485
Epoch 5, Train Loss: 0.0266, Valid MSE: 31.0193
Epoch 6, Train Loss: 0.0015, Valid MSE: 33.7135
Epoch 7, Train Loss: 0.0069, Valid MSE: 36.9831
Epoch 8, Train Loss: 0.0217, Valid MSE: 40.0034
Epoch 9, Train Loss: 0.0173, Valid MSE: 40.7441
Epoch 10, Train Loss: 0.0045, Valid MSE: 40.4281
Best Epoch: 0, Best Valid MSE: 15.823037147521973
Valid MSE (with VALID set) on Loaded Best Model: 15.823037147521973
[ outputs ]
tensor([[-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687],
        [-0.9687]

5-2. Probability Prediction

In [79]:
# Probability Prediction 용으로 변경

from torch.utils.data import Dataset

class MNIST_pp(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.class_map = {0: 1.0, 3: 1.0, 6: 1.0, 8: 1.0, 9: 1.0,
                          1: 0.0, 2: 0.0, 4: 0.0, 5: 0.0, 7: 0.0}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        new_label = self.class_map[label]
        return image, new_label

In [80]:
# Probability Prediction 용 Data Loader

train_loader_pp = DataLoader(MNIST_pp(train_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=True)

valid_loader_pp = DataLoader(MNIST_pp(valid_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=False)

test_loader_pp = DataLoader(MNIST_pp(test_loader.dataset),
                            batch_size=BATCH_SIZE,
                            shuffle=False)

In [81]:
analyze_class_distribution(train_loader_pp)

,class,count,percentage (%)
0,0.0,1007,50.35
1,1.0,993,49.65


In [82]:
analyze_class_distribution(valid_loader_pp)

,class,count,percentage (%)
0,0.0,1504,50.133333
1,1.0,1496,49.866667


In [83]:
analyze_class_distribution(test_loader_pp)

,class,count,percentage (%)
0,0.0,5069,50.69
1,1.0,4931,49.31


In [84]:
# Probability Prediction 실험 진행

_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.5525, Valid Accuracy: 0.8073
best model updated
Epoch 2, Train Loss: 0.3727, Valid Accuracy: 0.8640
best model updated
Epoch 3, Train Loss: 0.2702, Valid Accuracy: 0.9083
best model updated
Epoch 4, Train Loss: 0.1917, Valid Accuracy: 0.9173
best model updated
Epoch 5, Train Loss: 0.1393, Valid Accuracy: 0.9363
Epoch 6, Train Loss: 0.1178, Valid Accuracy: 0.9073
best model updated
Epoch 7, Train Loss: 0.0846, Valid Accuracy: 0.9510
Epoch 8, Train Loss: 0.0601, Valid Accuracy: 0.9450
Epoch 9, Train Loss: 0.0511, Valid Accuracy: 0.9490
best model updated
Epoch 10, Train Loss: 0.0365, Valid Accuracy: 0.9517
best model updated
Epoch 11, Train Loss: 0.0231, Valid Accuracy: 0.9530
best model updated
Epoch 12, Train Loss: 0.0216, Valid Accuracy: 0.9547
best model updated
Epoch 13, Train Loss: 0.0117, Valid Accuracy: 0.9617
Epoch 14, Train Loss: 0.0094, Valid Accuracy: 0.9567
best model updated
Epoch 15, Train Loss: 0.0098, Valid Accuracy: 0.9620
Epoch

In [85]:
_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=None)

best model updated
Epoch 1, Train Loss: 0.2444, Valid Accuracy: 0.7533
best model updated
Epoch 2, Train Loss: 0.1367, Valid Accuracy: 0.8590
best model updated
Epoch 3, Train Loss: 0.0896, Valid Accuracy: 0.8780
best model updated
Epoch 4, Train Loss: 0.0670, Valid Accuracy: 0.9330
best model updated
Epoch 5, Train Loss: 0.0505, Valid Accuracy: 0.9520
Epoch 6, Train Loss: 0.0455, Valid Accuracy: 0.9467
best model updated
Epoch 7, Train Loss: 0.0404, Valid Accuracy: 0.9580
best model updated
Epoch 8, Train Loss: 0.0310, Valid Accuracy: 0.9647
Epoch 9, Train Loss: 0.0278, Valid Accuracy: 0.9633
Epoch 10, Train Loss: 0.0233, Valid Accuracy: 0.9597
best model updated
Epoch 11, Train Loss: 0.0226, Valid Accuracy: 0.9670
best model updated
Epoch 12, Train Loss: 0.0189, Valid Accuracy: 0.9680
best model updated
Epoch 13, Train Loss: 0.0166, Valid Accuracy: 0.9720
Epoch 14, Train Loss: 0.0132, Valid Accuracy: 0.9713
Epoch 15, Train Loss: 0.0117, Valid Accuracy: 0.9700
Epoch 16, Train Loss: 0.

In [86]:
_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.1884, Valid Accuracy: 0.8113
best model updated
Epoch 2, Train Loss: 0.1187, Valid Accuracy: 0.8713
best model updated
Epoch 3, Train Loss: 0.0756, Valid Accuracy: 0.9073
best model updated
Epoch 4, Train Loss: 0.0506, Valid Accuracy: 0.9347
best model updated
Epoch 5, Train Loss: 0.0385, Valid Accuracy: 0.9377
best model updated
Epoch 6, Train Loss: 0.0264, Valid Accuracy: 0.9497
best model updated
Epoch 7, Train Loss: 0.0226, Valid Accuracy: 0.9550
best model updated
Epoch 8, Train Loss: 0.0138, Valid Accuracy: 0.9567
best model updated
Epoch 9, Train Loss: 0.0123, Valid Accuracy: 0.9587
best model updated
Epoch 10, Train Loss: 0.0102, Valid Accuracy: 0.9657
Epoch 11, Train Loss: 0.0079, Valid Accuracy: 0.9560
Epoch 12, Train Loss: 0.0060, Valid Accuracy: 0.9613
Epoch 13, Train Loss: 0.0042, Valid Accuracy: 0.9613
Epoch 14, Train Loss: 0.0038, Valid Accuracy: 0.9627
Epoch 15, Train Loss: 0.0037, Valid Accuracy: 0.9633
Epoch 16, Train Loss: 0.

In [87]:
_, best_epoch_model = run_experiment(nn.L1Loss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=None)

best model updated
Epoch 1, Train Loss: 0.3916, Valid Accuracy: 0.8240
best model updated
Epoch 2, Train Loss: 0.2505, Valid Accuracy: 0.8867
best model updated
Epoch 3, Train Loss: 0.2160, Valid Accuracy: 0.9210
Epoch 4, Train Loss: 0.1790, Valid Accuracy: 0.9193
best model updated
Epoch 5, Train Loss: 0.1612, Valid Accuracy: 0.9527
Epoch 6, Train Loss: 0.1398, Valid Accuracy: 0.9500
best model updated
Epoch 7, Train Loss: 0.1256, Valid Accuracy: 0.9637
Epoch 8, Train Loss: 0.1121, Valid Accuracy: 0.9630
best model updated
Epoch 9, Train Loss: 0.1111, Valid Accuracy: 0.9650
best model updated
Epoch 10, Train Loss: 0.1030, Valid Accuracy: 0.9653
Epoch 11, Train Loss: 0.0932, Valid Accuracy: 0.9650
Epoch 12, Train Loss: 0.0859, Valid Accuracy: 0.9647
best model updated
Epoch 13, Train Loss: 0.0807, Valid Accuracy: 0.9677
best model updated
Epoch 14, Train Loss: 0.0753, Valid Accuracy: 0.9680
Epoch 15, Train Loss: 0.0726, Valid Accuracy: 0.9657
Epoch 16, Train Loss: 0.0667, Valid Accurac

In [88]:
_, best_epoch_model = run_experiment(nn.L1Loss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.3789, Valid Accuracy: 0.6710
best model updated
Epoch 2, Train Loss: 0.2335, Valid Accuracy: 0.8183
best model updated
Epoch 3, Train Loss: 0.1655, Valid Accuracy: 0.8450
best model updated
Epoch 4, Train Loss: 0.1418, Valid Accuracy: 0.8690
best model updated
Epoch 5, Train Loss: 0.1287, Valid Accuracy: 0.8727
best model updated
Epoch 6, Train Loss: 0.1088, Valid Accuracy: 0.8883
best model updated
Epoch 7, Train Loss: 0.1044, Valid Accuracy: 0.8997
best model updated
Epoch 8, Train Loss: 0.0853, Valid Accuracy: 0.9130
best model updated
Epoch 9, Train Loss: 0.0663, Valid Accuracy: 0.9273
best model updated
Epoch 10, Train Loss: 0.0536, Valid Accuracy: 0.9320
best model updated
Epoch 11, Train Loss: 0.0402, Valid Accuracy: 0.9453
best model updated
Epoch 12, Train Loss: 0.0282, Valid Accuracy: 0.9503
Epoch 13, Train Loss: 0.0245, Valid Accuracy: 0.9483
best model updated
Epoch 14, Train Loss: 0.0219, Valid Accuracy: 0.9520
best model updated
E

In [89]:
_, best_epoch_model = run_experiment(RMSELoss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=None)

best model updated
Epoch 1, Train Loss: 0.4018, Valid Accuracy: 0.8750
best model updated
Epoch 2, Train Loss: 0.2803, Valid Accuracy: 0.9407
best model updated
Epoch 3, Train Loss: 0.2261, Valid Accuracy: 0.9523
best model updated
Epoch 4, Train Loss: 0.2009, Valid Accuracy: 0.9640
best model updated
Epoch 5, Train Loss: 0.1748, Valid Accuracy: 0.9680
best model updated
Epoch 6, Train Loss: 0.1550, Valid Accuracy: 0.9690
best model updated
Epoch 7, Train Loss: 0.1464, Valid Accuracy: 0.9730
Epoch 8, Train Loss: 0.1351, Valid Accuracy: 0.9710
best model updated
Epoch 9, Train Loss: 0.1163, Valid Accuracy: 0.9743
Epoch 10, Train Loss: 0.1048, Valid Accuracy: 0.9733
Epoch 11, Train Loss: 0.0969, Valid Accuracy: 0.9740
best model updated
Epoch 12, Train Loss: 0.0860, Valid Accuracy: 0.9757
best model updated
Epoch 13, Train Loss: 0.0793, Valid Accuracy: 0.9770
Epoch 14, Train Loss: 0.0776, Valid Accuracy: 0.9740
Epoch 15, Train Loss: 0.0678, Valid Accuracy: 0.9760
Epoch 16, Train Loss: 0.

In [90]:
_, best_epoch_model = run_experiment(RMSELoss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.4267, Valid Accuracy: 0.8210
best model updated
Epoch 2, Train Loss: 0.3164, Valid Accuracy: 0.8833
best model updated
Epoch 3, Train Loss: 0.2436, Valid Accuracy: 0.9180
best model updated
Epoch 4, Train Loss: 0.1899, Valid Accuracy: 0.9380
best model updated
Epoch 5, Train Loss: 0.1513, Valid Accuracy: 0.9507
Epoch 6, Train Loss: 0.1373, Valid Accuracy: 0.9300
best model updated
Epoch 7, Train Loss: 0.1104, Valid Accuracy: 0.9567
Epoch 8, Train Loss: 0.1029, Valid Accuracy: 0.9463
Epoch 9, Train Loss: 0.0914, Valid Accuracy: 0.9513
Epoch 10, Train Loss: 0.0646, Valid Accuracy: 0.9557
Epoch 11, Train Loss: 0.0935, Valid Accuracy: 0.9347
best model updated
Epoch 12, Train Loss: 0.0675, Valid Accuracy: 0.9583
best model updated
Epoch 13, Train Loss: 0.0376, Valid Accuracy: 0.9613
Epoch 14, Train Loss: 0.0345, Valid Accuracy: 0.9570
Epoch 15, Train Loss: 0.0566, Valid Accuracy: 0.9433
best model updated
Epoch 16, Train Loss: 0.0461, Valid Accurac

In [91]:
_, best_epoch_model = run_experiment(nn.CrossEntropyLoss(),
                                     train_loader=train_loader_pp,
                                     valid_loader=valid_loader_pp,
                                     test_loader=test_loader_pp,
                                     num_classes=1,
                                     task_type='prob_prediction',
                                     final_activation=nn.Softmax())

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 54.5276, Valid Accuracy: 0.4987
Epoch 2, Train Loss: 54.5496, Valid Accuracy: 0.4987
Epoch 3, Train Loss: 54.5276, Valid Accuracy: 0.4987
Epoch 4, Train Loss: 54.5936, Valid Accuracy: 0.4987
Epoch 5, Train Loss: 54.5386, Valid Accuracy: 0.4987
Epoch 6, Train Loss: 54.5496, Valid Accuracy: 0.4987
Epoch 7, Train Loss: 54.5606, Valid Accuracy: 0.4987
Epoch 8, Train Loss: 54.5166, Valid Accuracy: 0.4987
Epoch 9, Train Loss: 54.5496, Valid Accuracy: 0.4987
Epoch 10, Train Loss: 54.5166, Valid Accuracy: 0.4987
Best Epoch: 0, Best Valid Accuracy: 0.49866666666666665
Valid Accuracy (with VALID set) on Loaded Best Model: 0.49866666666666665
[ outputs ]
tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],


5-3. Binary Classification (2 outputs)

In [92]:
# Binary Classification 용으로 변경

from torch.utils.data import Dataset

class MNIST_bc(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.class_map = {0: 1, 3: 1, 6: 1, 8: 1, 9: 1,
                          1: 0, 2: 0, 4: 0, 5: 0, 7: 0}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        new_label = self.class_map[label]
        return image, new_label

In [93]:
# Binary Classification 용 Data Loader

train_loader_bc = DataLoader(MNIST_bc(train_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=True)

valid_loader_bc = DataLoader(MNIST_bc(valid_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=False)

test_loader_bc = DataLoader(MNIST_bc(test_loader.dataset),
                            batch_size=BATCH_SIZE,
                            shuffle=False)

In [94]:
analyze_class_distribution(train_loader_bc)

,class,count,percentage (%)
0,0,1007,50.35
1,1,993,49.65


In [95]:
analyze_class_distribution(valid_loader_bc)

,class,count,percentage (%)
0,0,1504,50.133333
1,1,1496,49.866667


In [96]:
analyze_class_distribution(test_loader_bc)

,class,count,percentage (%)
0,0,5069,50.69
1,1,4931,49.31


In [97]:
# Binary Classification 을 Class 2개 기준의 Categorical Cross-Entropy 를 이용하여 실험 진행

_, best_epoch_model = run_experiment(nn.CrossEntropyLoss(),
                                     train_loader=train_loader_bc,
                                     valid_loader=valid_loader_bc,
                                     test_loader=test_loader_bc,
                                     num_classes=2,
                                     final_activation=nn.Softmax())

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 0.5980, Valid Accuracy: 0.7900
best model updated
Epoch 2, Train Loss: 0.4785, Valid Accuracy: 0.8670
best model updated
Epoch 3, Train Loss: 0.4476, Valid Accuracy: 0.8777
best model updated
Epoch 4, Train Loss: 0.4062, Valid Accuracy: 0.9170
best model updated
Epoch 5, Train Loss: 0.3725, Valid Accuracy: 0.9330
best model updated
Epoch 6, Train Loss: 0.3558, Valid Accuracy: 0.9487
Epoch 7, Train Loss: 0.3512, Valid Accuracy: 0.9440
Epoch 8, Train Loss: 0.3390, Valid Accuracy: 0.9477
Epoch 9, Train Loss: 0.3338, Valid Accuracy: 0.9397
best model updated
Epoch 10, Train Loss: 0.3320, Valid Accuracy: 0.9570
Epoch 11, Train Loss: 0.3317, Valid Accuracy: 0.9560
best model updated
Epoch 12, Train Loss: 0.3247, Valid Accuracy: 0.9573
best model updated
Epoch 13, Train Loss: 0.3213, Valid Accuracy: 0.9610
Epoch 14, Train Loss: 0.3209, Valid Accuracy: 0.9600
best model updated
Epoch 15, Train Loss: 0.3218, Valid Accuracy: 0.9620
Epoch 16, Train Loss: 0.

In [98]:
_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader_bc,
                                     valid_loader=valid_loader_bc,
                                     test_loader=test_loader_bc,
                                     num_classes=2,
                                     final_activation=nn.Softmax())

best model updated
Epoch 1, Train Loss: 0.5890, Valid Accuracy: 0.7847
best model updated
Epoch 2, Train Loss: 0.3847, Valid Accuracy: 0.8663
best model updated
Epoch 3, Train Loss: 0.2640, Valid Accuracy: 0.9063
best model updated
Epoch 4, Train Loss: 0.1980, Valid Accuracy: 0.9250
best model updated
Epoch 5, Train Loss: 0.1416, Valid Accuracy: 0.9393
best model updated
Epoch 6, Train Loss: 0.1117, Valid Accuracy: 0.9450
best model updated
Epoch 7, Train Loss: 0.0753, Valid Accuracy: 0.9517
Epoch 8, Train Loss: 0.0554, Valid Accuracy: 0.9450
best model updated
Epoch 9, Train Loss: 0.0375, Valid Accuracy: 0.9523
Epoch 10, Train Loss: 0.0318, Valid Accuracy: 0.9123
Epoch 11, Train Loss: 0.0609, Valid Accuracy: 0.9407
best model updated
Epoch 12, Train Loss: 0.0312, Valid Accuracy: 0.9580
Epoch 13, Train Loss: 0.0203, Valid Accuracy: 0.9520
Epoch 14, Train Loss: 0.0172, Valid Accuracy: 0.9387
Epoch 15, Train Loss: 0.0079, Valid Accuracy: 0.9567
best model updated
Epoch 16, Train Loss: 0.

In [99]:
_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader_bc,
                                     valid_loader=valid_loader_bc,
                                     test_loader=test_loader_bc,
                                     num_classes=2,
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.5923, Valid Accuracy: 0.7953
best model updated
Epoch 2, Train Loss: 0.3780, Valid Accuracy: 0.8557
best model updated
Epoch 3, Train Loss: 0.2721, Valid Accuracy: 0.9053
best model updated
Epoch 4, Train Loss: 0.1980, Valid Accuracy: 0.9237
best model updated
Epoch 5, Train Loss: 0.1648, Valid Accuracy: 0.9397
best model updated
Epoch 6, Train Loss: 0.1098, Valid Accuracy: 0.9457
best model updated
Epoch 7, Train Loss: 0.0863, Valid Accuracy: 0.9567
Epoch 8, Train Loss: 0.0575, Valid Accuracy: 0.9547
Epoch 9, Train Loss: 0.0497, Valid Accuracy: 0.9563
best model updated
Epoch 10, Train Loss: 0.0320, Valid Accuracy: 0.9577
Epoch 11, Train Loss: 0.0322, Valid Accuracy: 0.9517
Epoch 12, Train Loss: 0.0327, Valid Accuracy: 0.9560
Epoch 13, Train Loss: 0.0179, Valid Accuracy: 0.9577
best model updated
Epoch 14, Train Loss: 0.0125, Valid Accuracy: 0.9640
Epoch 15, Train Loss: 0.0087, Valid Accuracy: 0.9560
Epoch 16, Train Loss: 0.0092, Valid Accurac

In [100]:
_, best_epoch_model = run_experiment(nn.BCEWithLogitsLoss(),
                                     train_loader=train_loader_bc,
                                     valid_loader=valid_loader_bc,
                                     test_loader=test_loader_bc,
                                     num_classes=2,
                                     final_activation=nn.Softmax())

best model updated
Epoch 1, Train Loss: 0.6698, Valid Accuracy: 0.7773
best model updated
Epoch 2, Train Loss: 0.5946, Valid Accuracy: 0.8470
best model updated
Epoch 3, Train Loss: 0.5677, Valid Accuracy: 0.8767
best model updated
Epoch 4, Train Loss: 0.5595, Valid Accuracy: 0.8907
best model updated
Epoch 5, Train Loss: 0.5519, Valid Accuracy: 0.9007
best model updated
Epoch 6, Train Loss: 0.5385, Valid Accuracy: 0.9157
best model updated
Epoch 7, Train Loss: 0.5334, Valid Accuracy: 0.9347
best model updated
Epoch 8, Train Loss: 0.5229, Valid Accuracy: 0.9397
Epoch 9, Train Loss: 0.5225, Valid Accuracy: 0.9220
best model updated
Epoch 10, Train Loss: 0.5179, Valid Accuracy: 0.9457
best model updated
Epoch 11, Train Loss: 0.5139, Valid Accuracy: 0.9543
best model updated
Epoch 12, Train Loss: 0.5117, Valid Accuracy: 0.9590
Epoch 13, Train Loss: 0.5093, Valid Accuracy: 0.9567
Epoch 14, Train Loss: 0.5088, Valid Accuracy: 0.9550
Epoch 15, Train Loss: 0.5081, Valid Accuracy: 0.9567
Epoch

In [101]:
_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_bc,
                                     valid_loader=valid_loader_bc,
                                     test_loader=test_loader_bc,
                                     num_classes=2,
                                     final_activation=nn.Softmax())

best model updated
Epoch 1, Train Loss: 0.1830, Valid Accuracy: 0.8257
best model updated
Epoch 2, Train Loss: 0.0990, Valid Accuracy: 0.8933
best model updated
Epoch 3, Train Loss: 0.0609, Valid Accuracy: 0.9267
best model updated
Epoch 4, Train Loss: 0.0394, Valid Accuracy: 0.9307
best model updated
Epoch 5, Train Loss: 0.0269, Valid Accuracy: 0.9393
best model updated
Epoch 6, Train Loss: 0.0198, Valid Accuracy: 0.9533
best model updated
Epoch 7, Train Loss: 0.0148, Valid Accuracy: 0.9547
best model updated
Epoch 8, Train Loss: 0.0132, Valid Accuracy: 0.9603
Epoch 9, Train Loss: 0.0093, Valid Accuracy: 0.9597
Epoch 10, Train Loss: 0.0108, Valid Accuracy: 0.9600
Epoch 11, Train Loss: 0.0040, Valid Accuracy: 0.9600
best model updated
Epoch 12, Train Loss: 0.0031, Valid Accuracy: 0.9673
Epoch 13, Train Loss: 0.0031, Valid Accuracy: 0.9667
Epoch 14, Train Loss: 0.0025, Valid Accuracy: 0.9653
Epoch 15, Train Loss: 0.0032, Valid Accuracy: 0.9557
Epoch 16, Train Loss: 0.0039, Valid Accurac

In [102]:
_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_bc,
                                     valid_loader=valid_loader_bc,
                                     test_loader=test_loader_bc,
                                     num_classes=2,
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.1868, Valid Accuracy: 0.7613
best model updated
Epoch 2, Train Loss: 0.1177, Valid Accuracy: 0.8587
best model updated
Epoch 3, Train Loss: 0.0898, Valid Accuracy: 0.8980
best model updated
Epoch 4, Train Loss: 0.0538, Valid Accuracy: 0.9110
best model updated
Epoch 5, Train Loss: 0.0408, Valid Accuracy: 0.9353
best model updated
Epoch 6, Train Loss: 0.0263, Valid Accuracy: 0.9383
best model updated
Epoch 7, Train Loss: 0.0239, Valid Accuracy: 0.9447
best model updated
Epoch 8, Train Loss: 0.0196, Valid Accuracy: 0.9453
best model updated
Epoch 9, Train Loss: 0.0141, Valid Accuracy: 0.9560
best model updated
Epoch 10, Train Loss: 0.0111, Valid Accuracy: 0.9623
Epoch 11, Train Loss: 0.0123, Valid Accuracy: 0.9573
Epoch 12, Train Loss: 0.0046, Valid Accuracy: 0.9580
Epoch 13, Train Loss: 0.0028, Valid Accuracy: 0.9593
best model updated
Epoch 14, Train Loss: 0.0019, Valid Accuracy: 0.9637
Epoch 15, Train Loss: 0.0018, Valid Accuracy: 0.9593
Epoch

5-4. Multi-Class Classification

In [103]:
# Multi-Class Classification (총 10개 확률 output)

_, best_epoch_model = run_experiment(nn.CrossEntropyLoss(),
                                     train_loader=train_loader,
                                     valid_loader=valid_loader,
                                     test_loader=test_loader,
                                     num_classes=10,
                                     final_activation=nn.Softmax())

best model updated
Epoch 1, Train Loss: 2.1582, Valid Accuracy: 0.7097
best model updated
Epoch 2, Train Loss: 1.8603, Valid Accuracy: 0.8397
best model updated
Epoch 3, Train Loss: 1.7079, Valid Accuracy: 0.8623
best model updated
Epoch 4, Train Loss: 1.6471, Valid Accuracy: 0.8643
best model updated
Epoch 5, Train Loss: 1.6151, Valid Accuracy: 0.8700
best model updated
Epoch 6, Train Loss: 1.5946, Valid Accuracy: 0.8717
best model updated
Epoch 7, Train Loss: 1.5706, Valid Accuracy: 0.9523
best model updated
Epoch 8, Train Loss: 1.5190, Valid Accuracy: 0.9597
best model updated
Epoch 9, Train Loss: 1.4995, Valid Accuracy: 0.9620
best model updated
Epoch 10, Train Loss: 1.4928, Valid Accuracy: 0.9670
Epoch 11, Train Loss: 1.4844, Valid Accuracy: 0.9633
best model updated
Epoch 12, Train Loss: 1.4781, Valid Accuracy: 0.9677
Epoch 13, Train Loss: 1.4750, Valid Accuracy: 0.9663
Epoch 14, Train Loss: 1.4745, Valid Accuracy: 0.9630
Epoch 15, Train Loss: 1.4721, Valid Accuracy: 0.9677
best 

In [104]:
_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader,
                                     valid_loader=valid_loader,
                                     test_loader=test_loader,
                                     num_classes=10,
                                     final_activation=nn.Softmax())

best model updated
Epoch 1, Train Loss: 0.2545, Valid Accuracy: 0.8323
best model updated
Epoch 2, Train Loss: 0.1325, Valid Accuracy: 0.9177
best model updated
Epoch 3, Train Loss: 0.0764, Valid Accuracy: 0.9460
Epoch 4, Train Loss: 0.0477, Valid Accuracy: 0.9383
best model updated
Epoch 5, Train Loss: 0.0330, Valid Accuracy: 0.9543
best model updated
Epoch 6, Train Loss: 0.0228, Valid Accuracy: 0.9570
best model updated
Epoch 7, Train Loss: 0.0180, Valid Accuracy: 0.9590
best model updated
Epoch 8, Train Loss: 0.0135, Valid Accuracy: 0.9650
best model updated
Epoch 9, Train Loss: 0.0095, Valid Accuracy: 0.9667
best model updated
Epoch 10, Train Loss: 0.0073, Valid Accuracy: 0.9713
Epoch 11, Train Loss: 0.0058, Valid Accuracy: 0.9690
Epoch 12, Train Loss: 0.0048, Valid Accuracy: 0.9687
Epoch 13, Train Loss: 0.0041, Valid Accuracy: 0.9697
Epoch 14, Train Loss: 0.0036, Valid Accuracy: 0.9700
Epoch 15, Train Loss: 0.0032, Valid Accuracy: 0.9703
Epoch 16, Train Loss: 0.0029, Valid Accurac

In [105]:
_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader,
                                     valid_loader=valid_loader,
                                     test_loader=test_loader,
                                     num_classes=10,
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.3930, Valid Accuracy: 0.0947
best model updated
Epoch 2, Train Loss: 0.3255, Valid Accuracy: 0.1053
Epoch 3, Train Loss: 0.3251, Valid Accuracy: 0.1053
Epoch 4, Train Loss: 0.3252, Valid Accuracy: 0.1053
Epoch 5, Train Loss: 0.3252, Valid Accuracy: 0.1053
Epoch 6, Train Loss: 0.3251, Valid Accuracy: 0.1053
Epoch 7, Train Loss: 0.3253, Valid Accuracy: 0.1053
Epoch 8, Train Loss: 0.3252, Valid Accuracy: 0.1053
Epoch 9, Train Loss: 0.3252, Valid Accuracy: 0.1053
Epoch 10, Train Loss: 0.3251, Valid Accuracy: 0.1053
best model updated
Epoch 11, Train Loss: 0.3251, Valid Accuracy: 0.1103
Epoch 12, Train Loss: 0.3251, Valid Accuracy: 0.1017
Epoch 13, Train Loss: 0.3254, Valid Accuracy: 0.1053
Epoch 14, Train Loss: 0.3252, Valid Accuracy: 0.1053
Epoch 15, Train Loss: 0.3250, Valid Accuracy: 0.0987
Epoch 16, Train Loss: 0.3251, Valid Accuracy: 0.1053
Epoch 17, Train Loss: 0.3253, Valid Accuracy: 0.0977
Epoch 18, Train Loss: 0.3253, Valid Accuracy: 0.105

In [106]:
_, best_epoch_model = run_experiment(nn.BCEWithLogitsLoss(),
                                     train_loader=train_loader,
                                     valid_loader=valid_loader,
                                     test_loader=test_loader,
                                     num_classes=10,
                                     final_activation=nn.Softmax())

best model updated
Epoch 1, Train Loss: 0.7226, Valid Accuracy: 0.7290
best model updated
Epoch 2, Train Loss: 0.6941, Valid Accuracy: 0.7813
best model updated
Epoch 3, Train Loss: 0.6786, Valid Accuracy: 0.9407
best model updated
Epoch 4, Train Loss: 0.6682, Valid Accuracy: 0.9520
best model updated
Epoch 5, Train Loss: 0.6634, Valid Accuracy: 0.9543
best model updated
Epoch 6, Train Loss: 0.6611, Valid Accuracy: 0.9577
best model updated
Epoch 7, Train Loss: 0.6598, Valid Accuracy: 0.9680
Epoch 8, Train Loss: 0.6585, Valid Accuracy: 0.9663
Epoch 9, Train Loss: 0.6578, Valid Accuracy: 0.9650
best model updated
Epoch 10, Train Loss: 0.6573, Valid Accuracy: 0.9683
Epoch 11, Train Loss: 0.6569, Valid Accuracy: 0.9667
Epoch 12, Train Loss: 0.6565, Valid Accuracy: 0.9667
Epoch 13, Train Loss: 0.6563, Valid Accuracy: 0.9673
best model updated
Epoch 14, Train Loss: 0.6561, Valid Accuracy: 0.9723
Epoch 15, Train Loss: 0.6560, Valid Accuracy: 0.9697
Epoch 16, Train Loss: 0.6559, Valid Accurac

In [107]:
_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader,
                                     valid_loader=valid_loader,
                                     test_loader=test_loader,
                                     num_classes=10,
                                     final_activation=nn.Softmax())

best model updated
Epoch 1, Train Loss: 0.0741, Valid Accuracy: 0.7760
best model updated
Epoch 2, Train Loss: 0.0354, Valid Accuracy: 0.9130
best model updated
Epoch 3, Train Loss: 0.0170, Valid Accuracy: 0.9390
best model updated
Epoch 4, Train Loss: 0.0104, Valid Accuracy: 0.9513
best model updated
Epoch 5, Train Loss: 0.0067, Valid Accuracy: 0.9610
best model updated
Epoch 6, Train Loss: 0.0046, Valid Accuracy: 0.9633
Epoch 7, Train Loss: 0.0033, Valid Accuracy: 0.9600
Epoch 8, Train Loss: 0.0024, Valid Accuracy: 0.9603
best model updated
Epoch 9, Train Loss: 0.0018, Valid Accuracy: 0.9650
best model updated
Epoch 10, Train Loss: 0.0013, Valid Accuracy: 0.9663
Epoch 11, Train Loss: 0.0009, Valid Accuracy: 0.9660
Epoch 12, Train Loss: 0.0008, Valid Accuracy: 0.9663
Epoch 13, Train Loss: 0.0007, Valid Accuracy: 0.9653
best model updated
Epoch 14, Train Loss: 0.0006, Valid Accuracy: 0.9667
Epoch 15, Train Loss: 0.0005, Valid Accuracy: 0.9667
Epoch 16, Train Loss: 0.0005, Valid Accurac

In [108]:
_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader,
                                     valid_loader=valid_loader,
                                     test_loader=test_loader,
                                     num_classes=10,
                                     final_activation=nn.Sigmoid())

best model updated
Epoch 1, Train Loss: 0.1110, Valid Accuracy: 0.0987
best model updated
Epoch 2, Train Loss: 0.0902, Valid Accuracy: 0.1053
Epoch 3, Train Loss: 0.0900, Valid Accuracy: 0.1053
Epoch 4, Train Loss: 0.0900, Valid Accuracy: 0.1053
Epoch 5, Train Loss: 0.0900, Valid Accuracy: 0.1053
Epoch 6, Train Loss: 0.0900, Valid Accuracy: 0.1053
Epoch 7, Train Loss: 0.0900, Valid Accuracy: 0.1053
Epoch 8, Train Loss: 0.0900, Valid Accuracy: 0.1053
Epoch 9, Train Loss: 0.0900, Valid Accuracy: 0.1053
Epoch 10, Train Loss: 0.0900, Valid Accuracy: 0.1053
Epoch 11, Train Loss: 0.0900, Valid Accuracy: 0.1053
Best Epoch: 1, Best Valid Accuracy: 0.10533333333333333
Valid Accuracy (with VALID set) on Loaded Best Model: 0.10533333333333333
[ outputs ]
tensor([[0.1055, 0.1210, 0.1046, 0.1070, 0.1046, 0.0952, 0.1001, 0.1015, 0.1008, 0.1152],
        [0.1055, 0.1210, 0.1046, 0.1070, 0.1046, 0.0952, 0.1001, 0.1015, 0.1008, 0.1152],
        [0.1055, 0.1210, 0.1046, 0.1070, 0.1046, 0.0952, 0.1001, 0

5-5. Multi-Label Classification (with 4 Labels)

In [109]:
# Multi-Label Classification 용으로 변경

from torch.utils.data import Dataset

class MNIST_mlc(Dataset):
    def __init__(self, dataset):
        self.dataset = dataset
        self.class_map = {0: torch.tensor([1., 0., 1., 1.]),
                          1: torch.tensor([0., 0., 0., 1.]),
                          2: torch.tensor([1., 1., 0., 0.]),
                          3: torch.tensor([0., 1., 1., 0.]),
                          4: torch.tensor([1., 0., 0., 1.]),
                          5: torch.tensor([0., 1., 0., 0.]),
                          6: torch.tensor([1., 0., 1., 0.]),
                          7: torch.tensor([0., 1., 0., 0.]),
                          8: torch.tensor([1., 0., 1., 0.]),
                          9: torch.tensor([0., 0., 1., 1.])}

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image, label = self.dataset[idx]
        new_label = self.class_map[label]
        return image, new_label

In [110]:
# Multi-Label Classification 용 Data Loader

train_loader_mlc = DataLoader(MNIST_mlc(train_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=True)

valid_loader_mlc = DataLoader(MNIST_mlc(valid_loader.dataset),
                             batch_size=BATCH_SIZE,
                             shuffle=False)

test_loader_mlc = DataLoader(MNIST_mlc(test_loader.dataset),
                            batch_size=BATCH_SIZE,
                            shuffle=False)

In [111]:
# 데이터셋 클래스 분포 분석 함수

def analyze_class_distribution_mlc(data_loader):
    targets = [str(data_loader.dataset[i][1]) for i in range(len(data_loader.dataset))]

    class_list, class_counts = np.unique(targets, return_counts=True)
    class_percentage = np.array(class_counts) * 100.0 / sum(class_counts)

    class_distrib = pd.DataFrame({'class': class_list,
                                  'count': class_counts,
                                  'percentage (%)': class_percentage})

    return class_distrib

In [112]:
analyze_class_distribution_mlc(train_loader_mlc)

,class,count,percentage (%)
0,"tensor([0., 0., 0., 1.])",239,11.95
1,"tensor([0., 0., 1., 1.])",211,10.55
2,"tensor([0., 1., 0., 0.])",372,18.60
3,"tensor([0., 1., 1., 0.])",214,10.70
4,"tensor([1., 0., 0., 1.])",200,10.00
5,"tensor([1., 0., 1., 0.])",382,19.10
6,"tensor([1., 0., 1., 1.])",186,9.30
7,"tensor([1., 1., 0., 0.])",196,9.80


In [113]:
analyze_class_distribution_mlc(valid_loader_mlc)

,class,count,percentage (%)
0,"tensor([0., 0., 0., 1.])",316,10.533333
1,"tensor([0., 0., 1., 1.])",296,9.866667
2,"tensor([0., 1., 0., 0.])",542,18.066667
3,"tensor([0., 1., 1., 0.])",331,11.033333
4,"tensor([1., 0., 0., 1.])",305,10.166667
5,"tensor([1., 0., 1., 0.])",581,19.366667
6,"tensor([1., 0., 1., 1.])",288,9.600000
7,"tensor([1., 1., 0., 0.])",341,11.366667


In [114]:
analyze_class_distribution_mlc(test_loader_mlc)

,class,count,percentage (%)
0,"tensor([0., 0., 0., 1.])",1135,11.35
1,"tensor([0., 0., 1., 1.])",1009,10.09
2,"tensor([0., 1., 0., 0.])",1920,19.20
3,"tensor([0., 1., 1., 0.])",1010,10.10
4,"tensor([1., 0., 0., 1.])",982,9.82
5,"tensor([1., 0., 1., 0.])",1932,19.32
6,"tensor([1., 0., 1., 1.])",980,9.80
7,"tensor([1., 1., 0., 0.])",1032,10.32


In [115]:
# 정상적으로 Binary Cross-Entropy 적용 시

# Sigmoid 를 이미 적용하여 최종 출력값을 0 ~ 1 로 만들었으므로,
# nn.BCEWithLogitsLoss() 가 아닌 nn.BCELoss() 를 사용

_, best_epoch_model = run_experiment(nn.BCELoss(),
                                     train_loader=train_loader_mlc,
                                     valid_loader=valid_loader_mlc,
                                     test_loader=test_loader_mlc,
                                     num_classes=4,
                                     final_activation=nn.Sigmoid(),
                                     multi_label=True)

best model updated
Epoch 1, Train Loss: 0.5939, Valid Accuracy: 0.8059
best model updated
Epoch 2, Train Loss: 0.4017, Valid Accuracy: 0.9038
best model updated
Epoch 3, Train Loss: 0.2829, Valid Accuracy: 0.9441
best model updated
Epoch 4, Train Loss: 0.2024, Valid Accuracy: 0.9617
best model updated
Epoch 5, Train Loss: 0.1472, Valid Accuracy: 0.9657
best model updated
Epoch 6, Train Loss: 0.1150, Valid Accuracy: 0.9664
best model updated
Epoch 7, Train Loss: 0.0964, Valid Accuracy: 0.9727
best model updated
Epoch 8, Train Loss: 0.0773, Valid Accuracy: 0.9731
Epoch 9, Train Loss: 0.0612, Valid Accuracy: 0.9678
best model updated
Epoch 10, Train Loss: 0.0529, Valid Accuracy: 0.9767
best model updated
Epoch 11, Train Loss: 0.0409, Valid Accuracy: 0.9783
Epoch 12, Train Loss: 0.0341, Valid Accuracy: 0.9782
best model updated
Epoch 13, Train Loss: 0.0279, Valid Accuracy: 0.9788
best model updated
Epoch 14, Train Loss: 0.0237, Valid Accuracy: 0.9794
Epoch 15, Train Loss: 0.0212, Valid Acc

In [116]:
_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_mlc,
                                     valid_loader=valid_loader_mlc,
                                     test_loader=test_loader_mlc,
                                     num_classes=4,
                                     final_activation=nn.Softmax(),
                                     multi_label=True)

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1736: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


best model updated
Epoch 1, Train Loss: 0.2358, Valid Accuracy: 0.6821
best model updated
Epoch 2, Train Loss: 0.1769, Valid Accuracy: 0.6996
best model updated
Epoch 3, Train Loss: 0.1535, Valid Accuracy: 0.7058
best model updated
Epoch 4, Train Loss: 0.1382, Valid Accuracy: 0.7125
best model updated
Epoch 5, Train Loss: 0.1305, Valid Accuracy: 0.7242
Epoch 6, Train Loss: 0.1246, Valid Accuracy: 0.7169
Epoch 7, Train Loss: 0.1205, Valid Accuracy: 0.7194
Epoch 8, Train Loss: 0.1193, Valid Accuracy: 0.7172
Epoch 9, Train Loss: 0.1159, Valid Accuracy: 0.7185
Epoch 10, Train Loss: 0.1147, Valid Accuracy: 0.7199
best model updated
Epoch 11, Train Loss: 0.1132, Valid Accuracy: 0.7432
best model updated
Epoch 12, Train Loss: 0.1122, Valid Accuracy: 0.7481
Epoch 13, Train Loss: 0.1115, Valid Accuracy: 0.7322
Epoch 14, Train Loss: 0.1110, Valid Accuracy: 0.7363
Epoch 15, Train Loss: 0.1109, Valid Accuracy: 0.7381
best model updated
Epoch 16, Train Loss: 0.1095, Valid Accuracy: 0.7506
Epoch 17,

In [117]:
_, best_epoch_model = run_experiment(nn.MSELoss(),
                                     train_loader=train_loader_mlc,
                                     valid_loader=valid_loader_mlc,
                                     test_loader=test_loader_mlc,
                                     num_classes=4,
                                     final_activation=nn.Sigmoid(),
                                     multi_label=True)

best model updated
Epoch 1, Train Loss: 0.1854, Valid Accuracy: 0.8558
best model updated
Epoch 2, Train Loss: 0.1032, Valid Accuracy: 0.9150
best model updated
Epoch 3, Train Loss: 0.0656, Valid Accuracy: 0.9519
best model updated
Epoch 4, Train Loss: 0.0441, Valid Accuracy: 0.9643
best model updated
Epoch 5, Train Loss: 0.0333, Valid Accuracy: 0.9704
Epoch 6, Train Loss: 0.0231, Valid Accuracy: 0.9667
Epoch 7, Train Loss: 0.0186, Valid Accuracy: 0.9665
best model updated
Epoch 8, Train Loss: 0.0144, Valid Accuracy: 0.9736
best model updated
Epoch 9, Train Loss: 0.0122, Valid Accuracy: 0.9758
best model updated
Epoch 10, Train Loss: 0.0093, Valid Accuracy: 0.9791
best model updated
Epoch 11, Train Loss: 0.0076, Valid Accuracy: 0.9793
best model updated
Epoch 12, Train Loss: 0.0058, Valid Accuracy: 0.9802
Epoch 13, Train Loss: 0.0052, Valid Accuracy: 0.9802
Epoch 14, Train Loss: 0.0049, Valid Accuracy: 0.9790
best model updated
Epoch 15, Train Loss: 0.0036, Valid Accuracy: 0.9808
Epoch

In [118]:
_, best_epoch_model = run_experiment(nn.CrossEntropyLoss(),
                                     train_loader=train_loader_mlc,
                                     valid_loader=valid_loader_mlc,
                                     test_loader=test_loader_mlc,
                                     num_classes=4,
                                     final_activation=nn.Softmax(),
                                     multi_label=True)

best model updated
Epoch 1, Train Loss: 2.3699, Valid Accuracy: 0.6942
best model updated
Epoch 2, Train Loss: 2.1957, Valid Accuracy: 0.7168
best model updated
Epoch 3, Train Loss: 2.1183, Valid Accuracy: 0.7340
best model updated
Epoch 4, Train Loss: 2.0775, Valid Accuracy: 0.7447
best model updated
Epoch 5, Train Loss: 2.0538, Valid Accuracy: 0.7502
Epoch 6, Train Loss: 2.0409, Valid Accuracy: 0.7482
Epoch 7, Train Loss: 2.0365, Valid Accuracy: 0.7458
best model updated
Epoch 8, Train Loss: 2.0262, Valid Accuracy: 0.7576
Epoch 9, Train Loss: 2.0225, Valid Accuracy: 0.7533
best model updated
Epoch 10, Train Loss: 2.0228, Valid Accuracy: 0.7578
best model updated
Epoch 11, Train Loss: 2.0153, Valid Accuracy: 0.7598
Epoch 12, Train Loss: 2.0124, Valid Accuracy: 0.7494
Epoch 13, Train Loss: 2.0144, Valid Accuracy: 0.7522
Epoch 14, Train Loss: 2.0059, Valid Accuracy: 0.7480
Epoch 15, Train Loss: 2.0053, Valid Accuracy: 0.7452
best model updated
Epoch 16, Train Loss: 2.0121, Valid Accurac